In [1]:
%pip  install -q condacolab   
import condacolab
condacolab.install_miniforge()


⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Miniforge3-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:22
🔁 Restarting kernel...


# Install Python libraries

Should be done via either pip or conda. We may already have them.

In [4]:
!conda install mdtraj ambertools=22

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - 

In [5]:
!cpptraj


CPPTRAJ: Trajectory Analysis. V6.4.4 (AmberTools)
    ___  ___  ___  ___
     | \/ | \/ | \/ | 
    _|_/\_|_/\_|_/\_|_

| Date/time: 05/16/23 19:41:49
| Available memory: 2.347 GB

^C


In [19]:
# Download the test trajectory
!wget -O- https://github.com/Amber-MD/pytraj/archive/refs/tags/v.2.0.5.tar.gz | tar -zxf-

--2023-05-16 20:29:56--  https://github.com/Amber-MD/pytraj/archive/refs/tags/v.2.0.5.tar.gz
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/Amber-MD/pytraj/tar.gz/refs/tags/v.2.0.5 [following]
--2023-05-16 20:29:56--  https://codeload.github.com/Amber-MD/pytraj/tar.gz/refs/tags/v.2.0.5
Resolving codeload.github.com (codeload.github.com)... 140.82.114.10
Connecting to codeload.github.com (codeload.github.com)|140.82.114.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘STDOUT’

-                       [<=>                 ]       0  --.-KB/s               pytraj-v.2.0.5/
pytraj-v.2.0.5/.gitignore
pytraj-v.2.0.5/.travis.yml
pytraj-v.2.0.5/CMakeLists.txt
pytraj-v.2.0.5/DEVELOPMENT.md
pytraj-v.2.0.5/Jenkinsfile
pytraj-v.2.0.5/MANIFEST.in
pytraj-v.2.0.5/README.m

In [20]:
!find . -name Tc5b.x

./pytraj-v.2.0.5/tests/data/Tc5b.x


# Pytraj

In [21]:
import pytraj as pt

# use iterload for memory saving
traj = pt.iterload("pytraj-v.2.0.5/tests/data/Tc5b.x", "pytraj-v.2.0.5/tests/data/Tc5b.top")

# calculate phi residue 3
dset = pt.calc_phi(traj, resrange='3-7')

# calcuate autocorrelation function for 1st dataset
af = pt.acorr(dset[0])
print(af)

[ 1.         -0.14098525  0.0815446  -0.33501382 -0.3415139   0.12924603
 -0.09997055  0.31578632 -0.112455    0.00336156]


In [22]:
dset

<pytraj.DatasetList with 5 datasets>
phi:3
[-149.03546384  -75.2000377   -78.25317579  -71.02248136  -63.91156199
 -116.60967828  -91.31475319 -132.49270182  -80.4173049   -95.86562288]

phi:4
[-155.79557499 -106.17617884 -134.02131493  -53.55778456  -59.58794832
  -94.98123448  -79.39922554  -64.5076182   -73.3429715   -81.34933091]

phi:5
[-125.82477671 -100.82317865 -118.23384293  -45.42351746  -65.64193036
  -80.093054    -88.08732265  -88.00621383 -108.76069876  -79.56164788]

phi:6
[-154.42303257 -148.64457702 -131.39734601  -82.5877152   -74.32836239
 -110.53379436 -102.63520333 -140.02184247  -68.3960965  -104.7528891 ]

phi:7
[-163.56723225 -146.44236323 -113.67087119 -134.3020574  -108.61681861
 -127.96071037 -122.54726782 -113.74723252 -140.51360719 -137.665533  ]

In [24]:
traj

pytraj.TrajectoryIterator, 10 frames: 
Size: 0.000068 (GB)
<Topology: 304 atoms, 20 residues, 1 mols, non-PBC>
           

# MDTraj

Note that MDTraj relies on file extensions so we symlink the files for loading.

In [30]:
import mdtraj as mdt
!ln -s pytraj-v.2.0.5/tests/data/Tc5b.top Tc5b.prmtop
!ln -s pytraj-v.2.0.5/tests/data/Tc5b.x Tc5b.mdcrd
traj_mdt = mdt.load("Tc5b.mdcrd", top="Tc5b.prmtop")

ln: failed to create symbolic link 'Tc5b.prmtop': File exists


In [31]:
traj_mdt

<mdtraj.Trajectory with 10 frames, 304 atoms, 20 residues, without unitcells at 0x7f81ce33b7c0>

In [33]:
phi_i, phi_a = mdt.compute_phi(traj_mdt)

In [39]:
phi_a.shape

(10, 19)

In [53]:
# Residue indices are different for some reason. The results match.
phi_subset = phi_a[:,1:6].T * 180/np.pi
phi_subset

array([[-149.03546 ,  -75.19999 ,  -78.253174, ..., -132.49269 ,
         -80.417305,  -95.865616],
       [-155.79549 , -106.17615 , -134.02132 , ...,  -64.50762 ,
         -73.34293 ,  -81.34929 ],
       [-125.82476 , -100.82319 , -118.23385 , ...,  -88.0062  ,
        -108.76068 ,  -79.561676],
       [-154.42303 , -148.64458 , -131.39734 , ..., -140.02182 ,
         -68.39611 , -104.75288 ],
       [-163.56723 , -146.44235 , -113.670876, ..., -113.74722 ,
        -140.51361 , -137.6655  ]], dtype=float32)

In [56]:
p0=phi_subset[0,:]

array([-149.03546 ,  -75.19999 ,  -78.253174,  -71.02249 ,  -63.911568,
       -116.609665,  -91.31482 , -132.49269 ,  -80.417305,  -95.865616],
      dtype=float32)

In [82]:
# ACF requires a couple of conversions
cor_p0=np.correlate(p0,p0,"full")
cor_p0=cor_p0[cor_p0.size//2:]
cor_p0

array([98266.734, 75752.4  , 71617.37 , 57600.703, 51168.227, 46452.18 ,
       36674.12 , 33295.277, 19194.123, 14287.376], dtype=float32)

In [80]:
# Pytraj/cpptraj use NORMALIZED ACF. 
p0s=(p0-p0.mean())/p0.std()
p0s

array([-1.9940327 ,  0.7516145 ,  0.63807875,  0.90695906,  1.171386  ,
       -0.78824645,  0.15236814, -1.378873  ,  0.55760336, -0.01685786],
      dtype=float32)

In [84]:
# The results match
cor_p0s=np.correlate(p0s,p0s,"full")
cor_p0s=cor_p0s[cor_p0s.size//2:]/len(p0s)
cor_p0s

array([ 1.0000001 , -0.14098492,  0.08154424, -0.3350137 , -0.3415141 ,
        0.12924582, -0.09997027,  0.31578642, -0.112455  ,  0.00336151],
      dtype=float32)